In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[3]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.00,0.000000,-9.152,0.0,0.0
1,7640.09,0.412318,-8.946,0.0,0.0
2,7391.37,0.398895,-8.676,0.0,0.0
3,7332.50,0.395718,-8.373,0.0,0.0
4,7002.01,0.377882,-8.140,0.0,0.0
...,...,...,...,...,...
43819,6911.08,0.372975,-0.174,0.0,0.0
43820,6638.85,0.358283,-0.627,0.0,0.0
43821,6412.14,0.346048,-0.723,0.0,0.0
43822,6451.50,0.348172,-0.659,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[3]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_CH,Temp_CH
0,0.00,-9.152
1,7640.09,-8.946
2,7391.37,-8.676
3,7332.50,-8.373
4,7002.01,-8.140
...,...,...
43819,6911.08,-0.174
43820,6638.85,-0.627
43821,6412.14,-0.723
43822,6451.50,-0.659


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_CH,Temp_CH,Lag-2,Lag-7
0,7563.04,-0.802,7536.97,0.00
1,7477.88,-0.745,7485.64,7640.09
2,7353.60,-0.743,7443.50,7391.37
3,7239.86,-0.902,7562.51,7332.50
4,7057.60,-0.991,7432.66,7002.01
...,...,...,...,...
43651,6911.08,-0.174,6872.48,6337.79
43652,6638.85,-0.627,6669.51,6145.57
43653,6412.14,-0.723,6513.77,6205.19
43654,6451.50,-0.659,6460.92,6645.40


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 23s 16ms/step - loss: 0.0031 - mape: 7122.6211 - mae: 0.0419 - val_loss: 0.0012 - val_mape: 9.7709 - val_mae: 0.0283
Epoch 2/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0024 - mape: 6862.4556 - mae: 0.0366 - val_loss: 0.0013 - val_mape: 10.4433 - val_mae: 0.0288
Epoch 3/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0018 - mape: 6378.0767 - mae: 0.0306 - val_loss: 0.0015 - val_mape: 9.9391 - val_mae: 0.0309
Epoch 4/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0016 - mape: 6330.2510 - mae: 0.0275 - val_loss: 0.0028 - val_mape: 15.0471 - val_mae: 0.0472
Epoch 5/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0015 - mape: 6342.1196 - mae: 0.0266 - val_loss: 0.0035 - val_mape: 16.6812 - val_mae: 0.0527
Epoch 6/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0014 - mape: 6287.9595 - mae: 0.0262 - val_los

1105/1105 [==============================] - 17s 16ms/step - loss: 9.9403e-04 - mape: 4711.4541 - mae: 0.0214 - val_loss: 0.0072 - val_mape: 24.5396 - val_mae: 0.0755
Epoch 48/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0010 - mape: 4913.2051 - mae: 0.0216 - val_loss: 0.0043 - val_mape: 18.2906 - val_mae: 0.0556
Epoch 49/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.9527e-04 - mape: 4892.3291 - mae: 0.0215 - val_loss: 0.0068 - val_mape: 23.9178 - val_mae: 0.0739
Epoch 50/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0010 - mape: 4938.3218 - mae: 0.0216 - val_loss: 0.0053 - val_mape: 21.0086 - val_mae: 0.0644
Epoch 51/100
1105/1105 [==============================] - 17s 16ms/step - loss: 9.8077e-04 - mape: 4720.1299 - mae: 0.0213 - val_loss: 0.0053 - val_mape: 20.5477 - val_mae: 0.0629
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0010 - mape: 4566.4922 - mae: 0.0216 - 

Epoch 93/100
1105/1105 [==============================] - 17s 15ms/step - loss: 8.7887e-04 - mape: 3134.2280 - mae: 0.0205 - val_loss: 0.0037 - val_mape: 16.4990 - val_mae: 0.0511
Epoch 94/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.1038e-04 - mape: 3607.5256 - mae: 0.0206 - val_loss: 0.0061 - val_mape: 22.5208 - val_mae: 0.0682
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.7244e-04 - mape: 2661.5872 - mae: 0.0197 - val_loss: 0.0055 - val_mape: 21.3254 - val_mae: 0.0642
Epoch 96/100
1105/1105 [==============================] - 17s 15ms/step - loss: 8.5126e-04 - mape: 3791.4255 - mae: 0.0202 - val_loss: 0.0044 - val_mape: 18.9221 - val_mae: 0.0584
Epoch 97/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.8155e-04 - mape: 2105.6140 - mae: 0.0197 - val_loss: 0.0037 - val_mape: 16.9399 - val_mae: 0.0518
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 7.5431e-04 - mape: 2

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1789.6310725212097


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 10ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.18149284, 0.18148564, 0.18593463, ..., 0.25480166, 0.25552678,
       0.25651166], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.23495642, 0.23426242, 0.2382909 , ..., 0.28913643, 0.29144545,
       0.31724778])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,5488.58,4577.232422
1,5476.75,4577.109375
2,5545.42,4652.948242
3,5443.58,4570.843750
4,5289.07,4658.024902
...,...,...
4363,6911.08,5815.622559
4364,6638.85,5818.473633
4365,6412.14,5826.865234
4366,6451.50,5839.225586


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.692620908851254